# Segmentace obrazu - text a jeho čtení
Cvičení je zaměřené na správné využití osvětlení při nasvícení objektu a následné využití metod pro segmentaci obrazu. Novinkou ve cvičení je využití OCR (optical character recognition) k převodu obrazu na text.

<img src="images/ocr.png" width="300">

Ke čtení je využita knihovna [**Tesseract OCR**](https://github.com/tesseract-ocr/tesseract).

### Import knihoven a konfigurace

In [ ]:
import os
import io

import cv2
import numpy as np
import matplotlib.pyplot as plt

from improutils import *
from pypylon_opencv_viewer import BaslerOpenCVViewer

%matplotlib inline
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})


### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu. Parametry a implementaci funkcí si můžete zobrazit pomocí příkazu `help(function_name)` nebo na https://gitlab.fit.cvut.cz/bi-svz/improutils_package/tree/master/improutils.

Seznam funkcí pro přehlednost:

- `to_gray(...)`
- `to_hsv(...)`
- `negative(...)`
- `crop(...)`


- `segmentation_one_threshold(...)`
- `segmentation_auto_threshold(...)`
- `segmentation_two_thresholds(...)`
- `segmentation_adaptive_threshold(...)`
- `find_contours(...)`


- `filtration_median(...)`
- `fill_holes(...)`


- `ocr(...)`


- `to_intensity(...)`

---

### Úkol

1. Zvolte vhodné funkce pro segmentaci obrazu a přečtěte text na obrazu snímaném kamerou. Po vyladění algoritmu, využijte obrázek z kamery a nechte ho přečíst OCR. V případě, že výsledek bude po porovnání `True`, jste úkol splnili.

2. Změřte velikost FIT kostičky v pixelech - výška, šířka.

Na každý úkol bude v ideálním případě potřeba jiný snímek. Volba vhodného nasvící usnadní algoritmickou část úkolu.

#### 1) Nasnímejte kostky FIT a zobrazte.
Snímky uložte do jedné složky. Použijte vhodné osvětlení!

In [ ]:
path = ... ### nezapomeňte cestu zakončit '/'
files = os.listdir(path)

images = []

for f in files:
    image = load_image(path + f) ### načtěte obrázek
    images.append(image)
        
print('Celkem nalezených obrázků: ' + str(len(images)))
plot_images(*images) #zobrazte snímky

#### 2) Doplňte funkční verze algoritmů pro nasvícení jednotlivých kostek FIT.
Výstupem každého algoritmu by měl být binární obraz, kde pozadí má černou barvu a objekty (popředí) bílou. Binární obrázek zobrazte.

**2a)** Funkce pro segmentaci jednoduchého referenčního obrazce (černo-bílý).

In [ ]:
def algorithm_easy(img):
    ###
    ...
    
    img = crop(...)
    
    ...

    return img_bin

In [ ]:
img_easy = images[...] ### doplňte index
img_easy_bin = algorithm_easy(img_easy)
plot_images(img_easy_bin)

**2b)** Funkce pro segmentaci těžšího referenčního obrazce.

In [ ]:
def algorithm_hard(img):
    ###
    
    ...
    
    img = crop(...)
    
    ...
    
    return img_bin

In [ ]:
img_hard = images[...] ### doplňte index
img_hard_bin = algorithm_hard(img_hard)
plot_images(img_hard_bin)

In [ ]:
images_preprocessed = [...] ### doplňte binární obrázky jednotlivých snímků

#### 3) Využijte funkční algoritmus na snímky.
**POZOR:** Dokud nebude vstupní obraz vypadat následovně, **NEPOUŠTĚJTE** se do další části.

<img src="images/fit_preprocessed.png" width="200">

#### 4) Využijte OCR na výsledný binární obrázek.

In [ ]:
texts = []
for image in images_preprocessed:
    text = ...(image) ###
    print(text)
    texts.append(text)

#### 5) Zkontrolujte správnost.

In [ ]:
ref_text = 'FIT'

for i, text in enumerate(texts):
    print('Přečtený text je: ' + text)
    if text.strip() == ref_text:
        print('-> Úkol ' + str(i+1) + ' jste splnili!\n')
    else:
        print('-> Úkol je třeba dál ladit ...')    
    print('')

#### 6) Změřte rozměry kostičky v pixelech

Úkolem ze snímku vysegmentovat kostičku a změřit její rozměry - výška, šířka v pixelech. Pokud se vám segmentace nedaří, nebojte se udělat snímek znovu či změnit nasvícení! Správné použití světel vám ulehčí hromadu práce.

Algoritmus zpočítá rozměry a vykreslí výsldek do původního snímku viz obrázek. Pro pozicování textu a obrysu kostičky využijte znalost kontury, ze které byly hodnoty vypočítány.

<img src="images/measures_ref.png" width="300">

In [ ]:
def algorithm_measures(img):
    
    ### algoritmus segmentace a měření, vrací masku, ze které byly rozměry počítané a vykreslení výsledků

    ...

    ###

    height, width = ... ###
    sizes_drawn = img.copy()
    points = ... ###

    cv2.line(sizes_drawn, points[0], points[1], color=(255, 0, 0), thickness=3)
    cv2.putText(sizes_drawn, "{:.2f} px".format(width), points[0], 0, 1.5, (0, 0, 255), 2)
    
    cv2.line(sizes_drawn, points[1], points[2], color=(255, 0, 0), thickness=3)
    cv2.putText(sizes_drawn, "{:.2f} px".format(height), points[2], 0, 1.5, (0, 0, 255), 2)

    return mask, sizes_drawn

In [ ]:
img_measures = images[...] #doplňte index
mask, sizes_drawn = algorithm_measures(img_measures)
plot_images(mask, sizes_drawn)